<a href="https://colab.research.google.com/github/xszaraz/oznal/blob/master/Copy_of_OZNAL_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
root_path = '/content/gdrive/My Drive/dataset'

In [0]:
!pip install pandas
!pip install numpy
!pip install ipython-autotime

%load_ext autotime

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats as sm_stats
import statsmodels.stats.api as sms
import math as mth

hospital_df = pd.read_csv("/content/gdrive/My Drive/dataset/hospital.csv")
pd.set_option("display.max_columns",None)
pd.set_option('max_colwidth', 999)
hospital_df.head()

In [0]:
hospital_df.loc[hospital_df['Total Costs'].between(0,10000), 'Total Costs'] = 1
hospital_df.loc[hospital_df['Total Costs'].between(10000.01,20000), 'Total Costs'] = 2
hospital_df.loc[hospital_df['Total Costs'] > 20000.01, 'Total Costs'] = 3

In [0]:
hospital_df['Total Costs'].value_counts()

In [0]:
hospital_df = hospital_df.drop(columns=['Ethnicity','Health Service Area','Hospital County','Facility Name', 'Facility Id', 'Zip Code - 3 digits', 'Operating Certificate Number', 'Discharge Year', 'CCS Diagnosis Code', 'CCS Procedure Code', 'APR DRG Code', 'APR MDC Code', 'APR Severity of Illness Code', 'Payment Typology 1', 'Payment Typology 2', 'Payment Typology 3', 'Total Charges'])

In [0]:
hospital_df

In [0]:
#check if some values contains null
hospital_df['Age Group'].isnull().values.any()
hospital_df['Gender'].isnull().values.any()
hospital_df['Race'].isnull().values.any()
hospital_df['Length of Stay'].isnull().values.any()
hospital_df['Type of Admission'].isnull().values.any()
hospital_df['Patient Disposition'].isnull().values.any()
hospital_df['CCS Diagnosis Description'].isnull().values.any()
hospital_df['CCS Procedure Description'].isnull().values.any()
hospital_df['APR DRG Description'].isnull().values.any()
hospital_df['APR MDC Description'].isnull().values.any()
hospital_df['APR Medical Surgical Description'].isnull().values.any()
hospital_df['Birth Weight'].isnull().values.any()
hospital_df['Abortion Edit Indicator'].isnull().values.any()
hospital_df['Emergency Department Indicator'].isnull().values.any()
hospital_df['Total Costs'].isnull().values.any()

In [0]:
#check how much values is missing
hospital_df['APR Severity of Illness Description'].isnull().sum()

In [0]:
#check how much values is missing
hospital_df['APR Risk of Mortality'].isnull().sum()

In [0]:
#check the rows with missing values 
hospital_df[hospital_df['APR Severity of Illness Description'].isnull()]

In [0]:
hospital_df['APR Severity of Illness Description'].unique()

In [0]:
hospital_df['APR Risk of Mortality'].unique()

In [0]:
#fill the NaN values 
hospital_df['APR Severity of Illness Description'].fillna( method ='ffill', inplace = True) 
hospital_df['APR Risk of Mortality'].fillna( method ='ffill', inplace = True) 

In [0]:
hospital_df['APR Severity of Illness Description'].isnull().values.any()
hospital_df['APR Risk of Mortality'].isnull().values.any()

In [0]:
hospital_df['CCS Diagnosis Description'].value_counts()

In [0]:
#replace samples with counts lower than 3500 with Other 
keys = hospital_df['CCS Diagnosis Description'].value_counts().keys()

for i in keys:
    if hospital_df['CCS Diagnosis Description'][hospital_df['CCS Diagnosis Description'] == i].count() <= 3500:
        hospital_df['CCS Diagnosis Description'] = hospital_df['CCS Diagnosis Description'].replace(i, "Other") 
        
hospital_df['CCS Diagnosis Description'].value_counts()

In [0]:
hospital_df['APR DRG Description'].value_counts()

In [0]:
#replace samples with counts lower than 3500 with Other 
keys = hospital_df['APR DRG Description'].value_counts().keys()

for i in keys:
    if hospital_df['APR DRG Description'][hospital_df['APR DRG Description'] == i].count() <= 3500:
        hospital_df['APR DRG Description'] = hospital_df['APR DRG Description'].replace(i, "Other") 
        
hospital_df['APR DRG Description'].value_counts()

In [0]:
hospital_df['APR MDC Description'].value_counts()

In [0]:
#We have 2 346 931 samples, but we dont want to predict on all of them because of the time its gonna take
 
#1368957 has value 1 and thats around 58 % of all samples from 2 346 931 samples
#548392 has value 2 and thats around 23 % all samples from 2 346 931 samples
#429582 has value 2 and thats around 19 %  all samples from 2 346 931 samples

#500 000 samples will form our new dataset for predicting and training
#290 000 samples in this new dataset will have value 1
#115 000 samples in this new dataset will have value 2
#95 000 samples in this new dataset will have value 3

#1000 samples will form our new dataset for finding best hyperparameters
#580 samples in this new dataset will have value 1
#230 samples in this new dataset will have value 1
#190 samples in this new dataset will have value 1

#for sample picking we will use dataset.sample()

dataset1 = hospital_df[hospital_df['Total Costs'] == 1]
to_merge1 = dataset1.sample(n = 290000)
to_hyperparameter1 = dataset1.sample(n = 580)
dataset2 = hospital_df[hospital_df['Total Costs'] == 2]
to_merge2 = dataset2.sample(n = 115000)
to_hyperparameter2 = dataset2.sample(n = 230)
dataset3 = hospital_df[hospital_df['Total Costs'] == 3]
to_merge3 = dataset3.sample(n = 95000)
to_hyperparameter3 = dataset3.sample(n = 190)

dataset_merged = pd.concat([to_merge1, to_merge2, to_merge3])
dataset_mergedhyperparameters = pd.concat([to_hyperparameter1, to_hyperparameter2, to_hyperparameter3])

In [0]:
#OneHotEncoding
dm_dummies1 = pd.get_dummies(dataset_merged['Age Group'])
dm_dummies2 = pd.get_dummies(dataset_merged['Gender'])
dm_dummies3 = pd.get_dummies(dataset_merged['Race'])
dm_dummies4 = pd.get_dummies(dataset_merged['Type of Admission'])
dm_dummies5 = pd.get_dummies(dataset_merged['Patient Disposition'])
dm_dummies6 = pd.get_dummies(dataset_merged['CCS Diagnosis Description'])
dm_dummies7 = pd.get_dummies(dataset_merged['CCS Procedure Description'])
dm_dummies8 = pd.get_dummies(dataset_merged['APR DRG Description'])
dm_dummies9 = pd.get_dummies(dataset_merged['APR MDC Description'])
dm_dummies10 = pd.get_dummies(dataset_merged['APR Severity of Illness Description'])
dm_dummies11 = pd.get_dummies(dataset_merged['APR Risk of Mortality'])
dm_dummies12 = pd.get_dummies(dataset_merged['APR Medical Surgical Description'])
dm_dummies13 = pd.get_dummies(dataset_merged['Abortion Edit Indicator'])
dm_dummies14 = pd.get_dummies(dataset_merged['Emergency Department Indicator'])

dmhp_dummies1 = pd.get_dummies(dataset_mergedhyperparameters['Age Group'])
dmhp_dummies2 = pd.get_dummies(dataset_mergedhyperparameters['Gender'])
dmhp_dummies3 = pd.get_dummies(dataset_mergedhyperparameters['Race'])
dmhp_dummies4 = pd.get_dummies(dataset_mergedhyperparameters['Type of Admission'])
dmhp_dummies5 = pd.get_dummies(dataset_mergedhyperparameters['Patient Disposition'])
dmhp_dummies6 = pd.get_dummies(dataset_mergedhyperparameters['CCS Diagnosis Description'])
dmhp_dummies7 = pd.get_dummies(dataset_mergedhyperparameters['CCS Procedure Description'])
dmhp_dummies8 = pd.get_dummies(dataset_mergedhyperparameters['APR DRG Description'])
dmhp_dummies9 = pd.get_dummies(dataset_mergedhyperparameters['APR MDC Description'])
dmhp_dummies10 = pd.get_dummies(dataset_mergedhyperparameters['APR Severity of Illness Description'])
dmhp_dummies11 = pd.get_dummies(dataset_mergedhyperparameters['APR Risk of Mortality'])
dmhp_dummies12 = pd.get_dummies(dataset_mergedhyperparameters['APR Medical Surgical Description'])
dmhp_dummies13 = pd.get_dummies(dataset_mergedhyperparameters['Abortion Edit Indicator'])
dmhp_dummies14 = pd.get_dummies(dataset_mergedhyperparameters['Emergency Department Indicator'])

In [0]:
#merging OneHotEncoding with the dataset from which OneHotEncodings were made
merged_df = pd.concat([dataset_merged, dm_dummies1, dm_dummies2, dm_dummies3, dm_dummies4, dm_dummies5, dm_dummies6, dm_dummies7, dm_dummies8, dm_dummies9, dm_dummies10, dm_dummies11, dm_dummies12, dm_dummies13, dm_dummies14], axis = 'columns')
merged_df_hp = pd.concat([dataset_mergedhyperparameters, dmhp_dummies1, dmhp_dummies2, dmhp_dummies3, dmhp_dummies4, dmhp_dummies5, dmhp_dummies6, dmhp_dummies7, dmhp_dummies8, dmhp_dummies9, dmhp_dummies10, dmhp_dummies11, dmhp_dummies12, dmhp_dummies13, dmhp_dummies14], axis = 'columns')

In [0]:
#dropping the original columns from which OneHotEncodings were made and replacing 120 + with 120 for training and predictioning 
features = merged_df.drop(columns=['Age Group', 'Gender', 'Race', 'Type of Admission', 'Patient Disposition', 'CCS Diagnosis Description', 'CCS Procedure Description', 'APR DRG Description', 'APR MDC Description', 'APR Severity of Illness Description', 'APR Risk of Mortality', 'APR Medical Surgical Description', 'Abortion Edit Indicator', 'Emergency Department Indicator'])
features['Length of Stay'] = features['Length of Stay'].replace('120 +', '120')

features2 = merged_df_hp.drop(columns=['Age Group', 'Gender', 'Race', 'Type of Admission', 'Patient Disposition', 'CCS Diagnosis Description', 'CCS Procedure Description', 'APR DRG Description', 'APR MDC Description', 'APR Severity of Illness Description', 'APR Risk of Mortality', 'APR Medical Surgical Description', 'Abortion Edit Indicator', 'Emergency Department Indicator'])
features2['Length of Stay'] = features2['Length of Stay'].replace('120 +', '120')

In [0]:
# Labels are the values we want to predict
labels = np.array(features['Total Costs'])
# Remove the labels from the features
# axis 1 refers to the columns
features = features.drop('Total Costs', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

# Labels2 are the values we want to predict
labels2 = np.array(features2['Total Costs'])
# Remove the labels from the features
# axis 1 refers to the columns
features2 = features2.drop('Total Costs', axis = 1)
# Saving feature names for later use
feature_list2 = list(features2.columns)
# Convert to numpy array
features2 = np.array(features2)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

In [0]:
#creating arrays for model comparising 
models = []
accuraccies = []
recalls = []
names = []
y_preds = []

In [0]:
#splitting datas for training, predicting and hyperparamaters searching
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state = 42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2, labels2, test_size = 0.25, random_state = 42)

In [0]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

X_train2 = X_train2.astype(int)
X_test2 = X_test2.astype(int)
y_train2 = y_train2.astype(int)
y_test2 = y_test2.astype(int)

In [0]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)
print('Training2 Features Shape:', X_train2.shape)
print('Training2 Labels Shape:', y_train2.shape)
print('Testing2 Features Shape:', X_test2.shape)
print('Testing2 Labels Shape:', y_test2.shape)

In [0]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [0]:
clf = RandomForestClassifier()
clf.get_params()

In [0]:
#GridSearchCV for RandomForestClassifier parameters

#The number of trees in the forest
n_estimators = [100, 300,700, 1200]
#The maximum depth of the tree
max_depth = [5, 10, 20, 40]
#The minimum number of samples required to split an internal node
min_samples_split = [2, 5, 10, 30, 100]
#The minimum number of samples required to be at a leaf node
min_samples_leaf = [1, 2, 5, 10]


param_grid = {'n_estimators': n_estimators,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf}

rf_grid = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
rf_grid.fit(X_train2, y_train2)
print(rf_grid.best_params_)

In [0]:
#RandomForestClassifier without optimalization
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='micro')
print("Accuracy: " + str(accuracy) + ", Recall(micro):" + str(recall) + "\n")

In [0]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [0]:
# Get numerical feature importances
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

for i in feature_importances:
    print("Variable: " + i[0] + ", Importance: " + str(i[1]))

In [0]:
models.append('RF')
accuraccies.append(accuracy)
recalls.append(recall)
y_preds.append(y_pred)

In [0]:
# RandomForestClassifier with GridSearchCV
clf_grid = RandomForestClassifier(n_estimators = 100, min_samples_split = 2, max_depth = 20, min_samples_leaf = 1)
clf_grid.fit(X_train, y_train)
y_pred = clf_grid.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='micro')
print("Accuracy: " + str(accuracy) + ", Recall(micro):" + str(recall) + "\n")

In [0]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [0]:
# Get numerical feature importances
importances = list(clf_grid.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

for i in feature_importances:
    print("Variable: " + i[0] + ", Importance: " + str(i[1]))

In [0]:
models.append('RF_grid')
accuraccies.append(accuracy)
recalls.append(recall)
y_preds.append(y_pred)

In [0]:
#GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
clf_gbm = GradientBoostingClassifier()
clf_gbm.get_params()

In [0]:
#GridSearchCV for GradientBoostingClassifier

#Loss function to be optimized. ‘deviance’ refers to deviance (= logistic regression) for classification with probabilistic outputs
loss = ['deviance']
#Learning rate shrinks the contribution of each tree by learning_rate. There is a trade-off between learning_rate and n_estimators
learning_rate = [1, 0.85, 0.5, 0.2, 0.05]
#The number of boosting stages to perform
n_estimators = [ 32, 64, 100, 200, 500]
#Maximum depth of the individual regression estimators
max_depth = [4, 6, 8, 10]
#The fraction of samples to be used for fitting the individual base learners
subsample = [0.6, 0.75, 0.85,0.9]

param_grid = {'loss': loss,
              'learning_rate': learning_rate,
              'n_estimators': n_estimators,
              'max_depth': max_depth,
              'subsample': subsample}

gbm_grid = GridSearchCV(estimator=clf_gbm, param_grid=param_grid, cv=5)
gbm_grid.fit(X_train2, y_train2)
print(gbm_grid.best_params_)

In [0]:
#GradientBoosting without optimalization
clf_gbm.fit(X_train, y_train)
y_pred = clf_gbm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='micro')
print("Accuracy: " + str(accuracy) + ", Recall(micro):" + str(recall) + "\n")

In [0]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [0]:
# Get numerical feature importances
importances = list(clf_gbm.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

for i in feature_importances:
    print("Variable: " + i[0] + ", Importance: " + str(i[1]))

In [0]:
models.append('GB')
accuraccies.append(accuracy)
recalls.append(recall)
y_preds.append(y_pred)

In [0]:
#GradientBoosting with GridSearchCV 
clf_gbm_grid = GradientBoostingClassifier(learning_rate = 0.05, loss = 'deviance',  max_depth = 6, n_estimators = 64, subsample = 0.75)
clf_gbm_grid.fit(X_train, y_train)
y_pred = clf_gbm_grid.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='micro')
print("Accuracy: " + str(accuracy) + ", Recall(micro):" + str(recall) + "\n")

In [0]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [0]:
# Get numerical feature importances
importances = list(clf_gbm_grid.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

for i in feature_importances:
    print("Variable: " + i[0] + ", Importance: " + str(i[1]))

In [0]:
models.append('GB_grid')
accuraccies.append(accuracy)
recalls.append(recall)
y_preds.append(y_pred)

In [0]:
#MLogisticRegression 
from sklearn import linear_model

In [0]:
lr = linear_model.LogisticRegression()
lr.get_params()

In [0]:
#GridSearch for LogisticRegression

#Used to specify the norm used in the penalization
penalty = ['l1', 'l2']
#Inverse of regularization strength
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
#Algorithm to use in the optimization problem, saga because it can handle both l1 and l2 penalty
solver = ['liblinear','saga']
#For multinomial the loss minimised is the multinomial loss fit across the entire probability distribution, even when the data is binary
multi_class = ['ovr']

param_grid = dict(penalty=penalty,
                  C=C,
                  solver=solver,
                  multi_class=multi_class)

mlr_grid = GridSearchCV(estimator = lr, param_grid=param_grid, verbose=1, n_jobs=-1, cv=5)
mlr_grid.fit(X_train2, y_train2)
print(mlr_grid.best_params_)

In [0]:
#lLogisticRegression without optimization
ovr_lr = linear_model.LogisticRegression(multi_class = 'ovr')
ovr_lr.fit(X_train, y_train)
y_pred = ovr_lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='micro')
print("Accuracy: " + str(accuracy) + ", Recall(micro):" + str(recall) + "\n")

In [0]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [0]:
models.append('OVR_lr')
accuraccies.append(accuracy)
recalls.append(recall)
y_preds.append(y_pred)

In [0]:
#lLogisticRegression with optimization
ovr_lr_grid = linear_model.LogisticRegression(multi_class = 'ovr', penalty = 'l2', solver = 'liblinear', C = 0.1)
ovr_lr_grid.fit(X_train, y_train)
y_pred = ovr_lr_grid.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='micro')
print("Accuracy: " + str(accuracy) + ", Recall(micro):" + str(recall) + "\n")

In [0]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [0]:
models.append('OVR_lr_grid')
accuraccies.append(accuracy)
recalls.append(recall)
y_preds.append(y_pred)

In [0]:
!pip install mord

In [0]:
import mord as m

In [0]:
ord_lr = m.LogisticIT()
ord_lr.fit(X_train, y_train)
y_pred = ord_lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='micro')
print("Accuracy: " + str(accuracy) + ", Recall(micro):" + str(recall) + "\n")

In [0]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [0]:
models.append('ORD_lr')
accuraccies.append(accuracy)
recalls.append(recall)
y_preds.append(y_pred)

In [0]:
from sklearn import svm, datasets
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import roc_auc_score
from itertools import cycle

In [0]:
len(models)

In [0]:
index = accuraccies.index(max(accuraccies))

In [0]:
y_pred_best = y_preds[index]

In [0]:
best_model = models[index]
best_model 

In [0]:
#This is needed for ROC curve
y_test_roc = label_binarize(y_test, classes=[1, 2, 3])
y_pred_roc = label_binarize(y_pred_best, classes=[1, 2, 3])

In [0]:
# Compute ROC curve and ROC area for each class
# Vypocitanie ROC curve a ROC oblasti pre kazdu triedu
lw = 2
n_classes = y_test_roc.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_roc[:, i], y_pred_roc[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_roc.ravel(), y_pred_roc.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [0]:
for i in range(len(models)):
    print("Model: " + models[i] + ", Accuracy: " + str(accuraccies[i]) + ", Recall(micro): " + str(recalls[i]))

In [0]:
objects = np.array(models)  
y_pos = np.arange(len(objects))
performance = np.array(accuraccies)  

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Accuracy')
plt.title('Accuraccies comparision')

plt.show()

In [0]:
objects = np.array(models)  
y_pos = np.arange(len(objects))
performance = np.array(recalls)  

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Recall')
plt.title('Recalls comparision')

plt.show()

In [0]:
#colleration
df = hospital_df
df['Age Group']=df['Age Group'].astype('category').cat.codes
df['Gender']=df['Gender'].astype('category').cat.codes
df['Race']=df['Race'].astype('category').cat.codes
df['Type of Admission']=df['Type of Admission'].astype('category').cat.codes
df['Patient Disposition']=df['Patient Disposition'].astype('category').cat.codes
df['CCS Diagnosis Description']=df['CCS Diagnosis Description'].astype('category').cat.codes
df['CCS Procedure Description']=df['CCS Procedure Description'].astype('category').cat.codes
df['APR DRG Description']=df['APR DRG Description'].astype('category').cat.codes
df['APR MDC Description']=df['APR MDC Description'].astype('category').cat.codes
df['APR Severity of Illness Description']=df['APR Severity of Illness Description'].astype('category').cat.codes
df['APR Risk of Mortality']=df['APR Risk of Mortality'].astype('category').cat.codes
df['APR Medical Surgical Description']=df['APR Medical Surgical Description'].astype('category').cat.codes
df['Abortion Edit Indicator']=df['Abortion Edit Indicator'].astype('category').cat.codes
df['Emergency Department Indicator']=df['Emergency Department Indicator'].astype('category').cat.codes
df.corr()